In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = 630
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'drive/MyDrive/tumor_dataset',
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
)

Found 253 files belonging to 2 classes.


In [ ]:
class_names = dataset.class_names

In [ ]:
train_ds = dataset.take(6)
val_ds = dataset.take(1)
test_ds = dataset.take(1)

In [ ]:
train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 630, 630, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_rescale,
    data_augmentation,
    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax')
])

# You may add more layers here (Lec 3)

model.build(input_shape=input_shape)

In [ ]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data = val_ds
)

Epoch 1/50
6/6 [==============================] - 69s 11s/step - loss: 85588.2969 - accuracy: 0.5052 - val_loss: 17954.8223 - val_accuracy: 0.4688
Epoch 2/50
6/6 [==============================] - 63s 10s/step - loss: 21623.1523 - accuracy: 0.6146 - val_loss: 20502.2324 - val_accuracy: 0.6250
Epoch 3/50
6/6 [==============================] - 61s 10s/step - loss: 11256.8643 - accuracy: 0.6406 - val_loss: 8136.7363 - val_accuracy: 0.5000
Epoch 4/50
6/6 [==============================] - 61s 10s/step - loss: 6564.1367 - accuracy: 0.5781 - val_loss: 5940.0942 - val_accuracy: 0.6875
Epoch 5/50
6/6 [==============================] - 61s 10s/step - loss: 3867.8027 - accuracy: 0.6875 - val_loss: 2052.4233 - val_accuracy: 0.7188
Epoch 6/50
6/6 [==============================] - 62s 10s/step - loss: 1717.1710 - accuracy: 0.7083 - val_loss: 877.6052 - val_accuracy: 0.6875
Epoch 7/50
6/6 [==============================] - 63s 10s/step - loss: 1216.5175 - accuracy: 0.6094 - val_loss: 691.8887 - val

In [ ]:
model.save('model_V1')

INFO:tensorflow:Assets written to: model_V1/assets
